In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# 123-D feature vectors
# 61 phoneme labels mapped to 39 classes
num_epochs = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 2
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length
dims = 123

In [3]:
def generateData():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1, 1))  # 3-D because tf.nn.dynamic_rnn only takes 3-D tensors
    y = y.reshape((batch_size, -1))

    return (x, y)

def generateSine():
    x = np.linspace(0, 100, total_series_length)
    y = np.cos(x)
    
    x = x.reshape((batch_size, -1, 1))  # 3-D because tf.nn.dynamic_rnn only takes 3-D tensors
    y = y.reshape((batch_size, -1))
    return x, y

x, y = generateSine()
print(x.shape)
print(y.shape)

(5, 10000, 1)
(5, 10000)


In [4]:
x = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length, 1], name="input_placeholder")
y = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length], name="label_placeholder")

# LSTM
# cell_state = tf.placeholder(tf.float32, [batch_size, state_size])
# hidden_state = tf.placeholder(tf.float32, [batch_size, state_size])
# init_state = tf.contrib.rnn.LSTMStateTuple(cell_state, hidden_state)

# RNN
init_state = tf.placeholder(tf.float32, [batch_size, state_size])


W = tf.Variable(np.random.rand(state_size, num_classes), dtype=tf.float32) # W is 4x2
b = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)# b is 1x2

# Unpack columns
labels_series = tf.unstack(y, axis=1)

# Forward passes

# RNN
cell = tf.contrib.rnn.BasicRNNCell(state_size)

#LSTM
# cell = tf.contrib.rnn.BasicLSTMCell(state_size, state_is_tuple=True)

states_series, current_state = tf.nn.dynamic_rnn(cell=cell, inputs=x, initial_state=init_state)

logits_series = [tf.matmul(state, W) + b for state in tf.unstack(states_series, axis=1)]
predictions_series = [tf.nn.softmax(logits) for logits in logits_series]

losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, labels in zip(logits_series,labels_series)]
total_loss = tf.reduce_mean(losses)

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)



In [5]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    loss_list = []

    for epoch_idx in range(num_epochs):
        x1,y1 = generateSine()
        # RNN
        _current_state = np.zeros((batch_size, state_size))
        
        # LSTM
#         _current_cell_state = np.zeros((batch_size, state_size))
#         _current_hidden_state = np.zeros((batch_size, state_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length

            batchX = x1[:,start_idx:end_idx]
            batchY = y1[:,start_idx:end_idx]
            
            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    x:batchX,
                    y:batchY,
                    # RNN
                    init_state: _current_state,
                    
                    # LSTM
#                     cell_state: _current_cell_state,
#                     hidden_state: _current_hidden_state
                })

            loss_list.append(_total_loss)

            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", _total_loss)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
New data, epoch 0
Step 0 Loss 0.810115
Step 100 Loss 0.00175132
Step 200 Loss 0.000897138
Step 300 Loss 0.000606472
Step 400 Loss 0.000458396
Step 500 Loss 0.000368527
Step 600 Loss 0.000308109
New data, epoch 1
Step 0 Loss 0.0394616
Step 100 Loss 0.000246571
Step 200 Loss 0.000214125
Step 300 Loss 0.000191956
Step 400 Loss 0.00017415
Step 500 Loss 0.00015937
Step 600 Loss 0.000146974
New data, epoch 2
Step 0 Loss 0.0408375
Step 100 Loss 0.000133803
Step 200 Loss 0.00012242
Step 300 Loss 0.000114816
Step 400 Loss 0.000108117
Step 500 Loss 0.000102276
Step 600 Loss 9.70317e-05
New data, epoch 3
Step 0 Loss 0.0408072
Step 100 Loss 9.23209e-05
Step 200 Loss 8.61131e-05
Step 300 Loss 8.21557e-05
Step 400 Loss 7.8675e-05
Step 500 Loss 7.55758e-05
Step 600 Loss 7.25958e-05
New data, epoch 4
Step 0 Loss 0.0401219
Step 100 Loss 7.06711e-05
Step 200 Loss 6.65404e-05
Step 300 Loss 6.41564e-05
Step 400 Loss 6.19869e-05
Step

Step 200 Loss 6.69954e-06
Step 300 Loss 6.6757e-06
Step 400 Loss 6.6757e-06
Step 500 Loss 6.6757e-06
Step 600 Loss 6.6757e-06
New data, epoch 41
Step 0 Loss 0.0163354
Step 100 Loss 6.81875e-06
Step 200 Loss 6.58033e-06
Step 300 Loss 6.55649e-06
Step 400 Loss 6.55649e-06
Step 500 Loss 6.43728e-06
Step 600 Loss 6.43728e-06
New data, epoch 42
Step 0 Loss 0.0160455
Step 100 Loss 6.67888e-06
Step 200 Loss 6.34191e-06
Step 300 Loss 6.31807e-06
Step 400 Loss 6.31807e-06


KeyboardInterrupt: 